## Task augmentation

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torchvision.models import resnet50
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torch.nn.functional as F
from tqdm.auto import tqdm,trange
from pathlib import Path
import pandas as pd
import torch.nn.utils.prune as prune
import numpy as np
import random
import learn2learn as l2l
import pickle
import matplotlib.pyplot as plt

/home/samuel/.conda/envs/meta/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
ways = 5 
shots =5

In [4]:
with open("./data/test.pkl",'rb') as f:
    test_data = pickle.load(f)
with open("./data/train.pkl",'rb') as f:
    train_data = pickle.load(f)
with open("./data/validation.pkl",'rb') as f:
    val_data = pickle.load(f)

In [5]:
train_data['images'].shape

(38400, 3, 84, 84)

In [6]:
val_data['images'].shape

(9600, 3, 84, 84)

# Task augmentation

In [7]:
%%time
images = train_data["images"]
labels = train_data["labels"]
num_labels = np.unique(labels).shape[0]
num_new_labels = num_labels*3
n_data = int(len(labels)/num_labels)
for i in range(num_labels):
    images_of_one_class = images[i*n_data:(i+1)*n_data] #600
    new_img1 = np.array([np.rot90(img.copy(), axes=(2,1)) for img in images_of_one_class]) 
    new_img2 = np.array([np.rot90(img.copy(),2, axes=(2,1)) for img in images_of_one_class])
    new_img3 = np.array([np.rot90(img.copy(),3, axes=(2,1)) for img in images_of_one_class])
    train_data["images"] = np.concatenate((train_data["images"],new_img1,new_img2,new_img3))
    train_data["labels"] = np.concatenate((train_data["labels"],[num_labels+3*i]*n_data,[num_labels+3*i+1]*n_data,[num_labels+3*i+2]*n_data))
    
    # print(train_data['labels'].shape)
    # break

CPU times: user 31 s, sys: 38.4 s, total: 1min 9s
Wall time: 1min 9s


In [8]:
train_data["images"].shape,train_data["labels"].shape

((153600, 3, 84, 84), (153600,))

In [11]:
%%time
images = val_data["images"]
labels = val_data["labels"]
num_labels = np.unique(labels).shape[0]
num_new_labels = num_labels*3
n_data = int(len(labels)/num_labels)
for i in range(num_labels):
    images_of_one_class = images[i*n_data:(i+1)*n_data]
    new_img1 = np.array([np.rot90(img.copy(), axes=(2,1)) for img in images_of_one_class]) 
    new_img2 = np.array([np.rot90(img.copy(),2, axes=(2,1)) for img in images_of_one_class])
    new_img3 = np.array([np.rot90(img.copy(),3, axes=(2,1)) for img in images_of_one_class])
    val_data["images"] = np.concatenate((val_data["images"],new_img1,new_img2,new_img3))
    val_data["labels"] = np.concatenate((val_data["labels"],[num_labels+3*i]*n_data,[num_labels+3*i+1]*n_data,[num_labels+3*i+2]*n_data))

CPU times: user 2.33 s, sys: 3.12 s, total: 5.45 s
Wall time: 5.45 s


In [12]:
val_data["images"].shape,val_data["labels"].shape

((38400, 3, 84, 84), (38400,))

In [13]:
class Dataset(Dataset):
    def __init__(self,data,transform = None):
        self.transform = transform
        self.data=data
        self.images = np.transpose(self.data['images'],(0,2,3,1))
        self.labels = data['labels']
    def __len__(self): 
        return len(self.data['images'])
    def __getitem__(self,index):
        img = self.images[index]
        label = self.labels[index]
        if self.transform:
            img = self.transform(img)
        return img, label

In [14]:
train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(degrees = 0, translate=(0.1,0.1)),
    ])
val_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomAffine(degrees = 0, translate=(0.1,0.1))
])

In [15]:
train_dataset = Dataset(train_data,train_transform)
val_dataset = Dataset(val_data, val_transform)

In [16]:
train_dataset = l2l.data.MetaDataset(train_dataset)
val_dataset = l2l.data.MetaDataset(val_dataset)

In [17]:
task_transforms = [
    l2l.data.transforms.FusedNWaysKShots(train_dataset, n=ways, k=2*shots),
    l2l.data.transforms.LoadData(train_dataset),
    l2l.data.transforms.RemapLabels(train_dataset),
    l2l.data.transforms.ConsecutiveLabels(train_dataset),
    # l2l.vision.transforms.RandomClassRotation(train_dataset,degrees = [0,10,15,20])
]
train_taskset =  l2l.data.TaskDataset(train_dataset, task_transforms, num_tasks=20000)
task_transforms = [
    l2l.data.transforms.FusedNWaysKShots(val_dataset, n = ways, k=2*shots),
    l2l.data.transforms.LoadData(val_dataset),
    l2l.data.transforms.RemapLabels(val_dataset),
    l2l.data.transforms.ConsecutiveLabels(val_dataset),
    # l2l.vision.transforms.RandomClassRotation(val_dataset,degrees = [0,10,15,20])
]
val_taskset =  l2l.data.TaskDataset(val_dataset, task_transforms, num_tasks=20000)

In [19]:
def adaptation(task, learner, fas,loss_func, device):
    data, labels = task
    data = data.to(device)
    labels = labels.to(device)
    sup_mask = np.array(([False]*5+[True]*5)*5)
    query_mask = ~sup_mask
    sup_data , query_data = data[sup_mask], data[query_mask]
    sup_labels , query_labels = labels[sup_mask], labels[query_mask]
    for step in range(fas): # inner loop
        pred = learner(sup_data)
        train_loss = loss_func(pred, sup_labels)
        learner.adapt(train_loss)
    query_pred = learner(query_data)
    query_loss = loss_func(query_pred,query_labels)
    query_acc = (torch.argmax(query_pred,1)==query_labels).sum()/len(query_labels)
    return query_loss, query_acc

def train(EPOCHS, meta_model, train_taskset, val_taskset, loss_func, opt, device, model_name):
    log_file = f"./log/{model_name}.txt"
    res = {
        "train_acc_list" : [],
        "val_acc_list" : [],
        "train_loss_list" : [],
        "val_loss_list" : [],
    }
    last_val_loss=float("inf")
    for epoch in trange(EPOCHS):
        opt.zero_grad()
        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0
        for _ in range(meta_batch):
            learner = meta_model.clone(first_order = False)
            task = train_taskset.sample() # sample 一組 [50,3,84,84],[50]
            query_loss, query_acc = adaptation(task, learner,fas, loss_func, device)
            query_loss.backward()
            train_loss += query_loss.item()
            train_acc += query_acc.item()
            #validation
            learner = meta_model.clone(first_order = False)
            task = val_taskset.sample() # sample 一組 [50,3,84,84],[50]
            query_loss, query_acc = adaptation(task, learner,fas, loss_func, device)
            val_loss+=query_loss.item()
            val_acc +=query_acc.item()
        res["train_acc_list"].append((train_acc/meta_batch))
        res["val_acc_list"].append((val_acc/meta_batch))
        res["train_loss_list"].append((train_loss/meta_batch))
        res["val_loss_list"].append((val_loss/meta_batch))
        for p in meta_model.parameters():
            p.grad.data.mul_(1.0 / meta_batch)
        opt.step()
        
        # write log file
        out = {
            "Epoch" : epoch,
            "train accuracy" : (train_acc/meta_batch),
            "validation accuracy" : (val_acc/meta_batch),
            "train loss" : (train_loss/meta_batch),
            "validation loss" : (val_loss/meta_batch)
        }
        with open(log_file,"a") as f:
            f.write(str(out) + '\n')
            
        # print log
        if (epoch+1)%100==0:
            print(f"Epoch {epoch+1} | Train loss :{train_loss/meta_batch} | Train accuracy : {train_acc/meta_batch}")
            print(f"Epoch {epoch+1} | Validation loss :{val_loss/meta_batch} | Validation accuracy : {val_acc/meta_batch}")
        # save model
        if (epoch+1)%500==0:
            MODEL_PATH=f"./model/{model_name}"
            Path(MODEL_PATH).mkdir(parents=True, exist_ok=True)
            torch.save(meta_model,Path(MODEL_PATH)/f"model_{epoch+1}.pt")
        # early stop
        n = 750
        if len(res["val_loss_list"])>n:
            if (epoch+1)%n == 0:
                avg_val_loss = sum(res["val_loss_list"][-n:])/n
                print("Average val_loss of last 500 epoch:",avg_val_loss)
                if last_val_loss < avg_val_loss:
                    print("Early stop!!!")
                    return res
                last_val_loss = avg_val_loss
    return res

In [20]:
def test(meta_model, test_data, loss_func, device):
    test_loss = 0
    test_acc = 0
    test_sup_images = test_data['sup_images']
    test_sup_labels = test_data['sup_labels']
    test_qry_images = test_data['qry_images']
    all_pred = []
    for i in trange(len(test_sup_images)):
        learner = meta_model.clone(first_order = False)
        sup_image = torch.tensor(test_sup_images[i]).to(device)
        sup_label = torch.tensor(test_sup_labels[i]).to(device)
        qry_image = torch.tensor(test_qry_images[i]).to(device)
        for step in range(fas): # inner loop
            pred = learner(sup_image)
            train_loss = loss_func(pred, sup_label)
            learner.adapt(train_loss)
        output_pred = torch.argmax(learner(qry_image),1).reshape(-1,1).detach().cpu().numpy()
        all_pred += [item for sublist in output_pred for item in sublist]
    return all_pred

In [ ]:
maml_lr = 0.5
lr = 1e-3
EPOCHS = 12000
meta_batch = 32
fas = 1
model = l2l.vision.models.ResNet12(output_size=ways).to(device)
meta_model = l2l.algorithms.MAML(model, lr=maml_lr,first_order=False)
opt = torch.optim.AdamW(meta_model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss(reduction='mean')
results  = train(EPOCHS, meta_model, train_taskset, val_taskset, loss_func,opt, device, "resnet_task_augmentation_v2")

  1%|▏                  | 100/12000 [33:52<67:06:29, 20.30s/it]

Epoch 100 | Train loss :1.570712849497795 | Train accuracy : 0.32874999032355845
Epoch 100 | Validation loss :1.5530204735696316 | Validation accuracy : 0.34249999141320586


  2%|▎                | 200/12000 [1:07:39<66:30:00, 20.29s/it]

Epoch 200 | Train loss :1.38582044839859 | Train accuracy : 0.40624998742714524
Epoch 200 | Validation loss :1.4666130542755127 | Validation accuracy : 0.3537499886006117


  2%|▍                | 300/12000 [1:41:26<65:55:01, 20.28s/it]

...

 32%|████▉          | 3900/12000 [22:10:23<45:46:10, 20.34s/it]

Epoch 3900 | Train loss :0.6313612057128921 | Train accuracy : 0.7687499774619937
Epoch 3900 | Validation loss :0.91527886595577 | Validation accuracy : 0.6299999840557575


 33%|█████          | 4000/12000 [22:44:18<45:15:11, 20.36s/it]

Epoch 4000 | Train loss :0.4645992508158088 | Train accuracy : 0.8162499740719795
Epoch 4000 | Validation loss :0.8625044822692871 | Validation accuracy : 0.6562499888241291


 34%|█████          | 4096/12000 [23:16:54<44:43:25, 20.37s/it]

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w
df1 = pd.DataFrame(results["train_acc_list"],columns=['train_accuracy'])
df2 = pd.DataFrame([i for i in results["val_acc_list"]],columns=['val_accuracy'])
df = pd.concat([df1,df2],axis=1)
df.plot()

In [ ]:
df1 = pd.DataFrame(results["train_loss_list"][1000:],columns=['train_loss'])
df2 = pd.DataFrame([i for i in results["val_loss_list"][1000:]],columns=['val_loss'])
df = pd.concat([df1,df2],axis=1)
df.plot()